### Loading the Necessary Github Repos
* <b> Please double check if git repos are cloned properly, otherwise notebook will run into multiple issues.</b>

In [ ]:
!pip install onnx
!pip install onnx_tf
!git clone https://github.com/usefulsensors/openai-whisper.git
!git clone https://github.com/openai/whisper.git

In [ ]:
%%bash
cp whisper.patch whisper/
cd whisper/
patch -p1 < ./whisper.patch

### Installing Necessary Libraries

In [ ]:
!pip3 install transformers==4.29.2 
!pip3 install safetensors==0.3.0
!pip3 install pyyaml==5.3
!pip3 install numpy==1.22.2
!pip3 install torchvision==0.15.2
!pip3 install packaging==21.3
!pip3 install datasets
!pip3 install protobuf==3.20 --force-reinstall
!pip3 install tiktoken numba tqdm more-itertools triton>=2.0.0

In [ ]:
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"]="python"

In [ ]:
import whisper
import torch
#import tensorflow as tf
import onnx
import numpy as np
import argparse
import os
import warnings
import tqdm
#from onnx_tf.backend import prepare
from whisper.whisper import load_model
from whisper.whisper.audio import load_audio, log_mel_spectrogram,pad_or_trim,N_FRAMES, SAMPLE_RATE
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

#load openai->whisper(pytorch)->tiny model
tiny_model = load_model("tiny")

#Export to onnx format
torch.onnx.export(tiny_model.encoder,torch.randn(1,80,3000).to(device), "./whisper_encoder.onnx",opset_version=14)

### Getting the Encoder ONNX Model

### IDEA
**Basic Idea will be**
- Encoder will run as dlc
- Decoder will run as tflite Model
- The output generated from decoder will be added with output from previous decoder instance and then multiplied with the encoder result

![modelArchitecture](image-assets/whisper_model_Architecture.png)



#### Normal Model Inference

In [1]:
#!pip3 install torch==1.8.1 
!pip3 install onnxruntime
!pip3 install tensorflow==2.10.1
!pip3 install tflite==2.3.0
!pip3 install soundfile
!pip3 install librosa
!pip3 install numpy==1.26.4
!pip3 install "protobuf==3.20" --force-reinstall

ERROR: Could not find a version that satisfies the requirement torch==1.8.1 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.7.1)
ERROR: No matching distribution found for torch==1.8.1
  Using cached protobuf-3.19.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (787 bytes)
Using cached protobuf-3.19.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.0
    Uninstalling protobuf-3.20.0:
      Successfully uninstalled protobuf-3.20.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
onnx 1.16.1 requires protobuf>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
  Using cached protobuf-3.20.0-cp310-cp310-manylinux_2_12

In [2]:
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"]="python"

In [3]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset
import torch

# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")


# load dummy dataset and read audio files
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
sample = ds[0]["audio"]
input_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt") 
# generate token ids
predicted_ids = model.generate(**input_features,decoder_input_ids=torch.tensor([[50258, 50259, 50359, 50363]]))
# decode token ids to text

transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
print(transcription)

/usr/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/venv/lib/python3.10/site-packages/transformers/modeling_utils.py:433: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(checkpoint_file, framework="pt") as f:
/usr/venv/lib/python3.10/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "cipher": algorithms.TripleDES,
/us

[' Mr. Quilter is the apostle of the middle classes and we are glad to welcome his gospel.']


#### Getting The Special Token Values

In [4]:
special_tokens=['<|endoftext|>', '<|startoftranscript|>', '<|en|>', '<|zh|>', '<|de|>', '<|es|>', '<|ru|>', '<|ko|>', '<|fr|>', '<|ja|>', '<|pt|>', '<|tr|>', '<|pl|>', '<|ca|>', '<|nl|>', '<|ar|>', '<|sv|>', '<|it|>', '<|id|>', '<|hi|>', '<|fi|>', '<|vi|>', '<|he|>', '<|uk|>', '<|el|>', '<|ms|>', '<|cs|>', '<|ro|>', '<|da|>', '<|hu|>', '<|ta|>', '<|no|>', '<|th|>', '<|ur|>', '<|hr|>', '<|bg|>', '<|lt|>', '<|la|>', '<|mi|>', '<|ml|>', '<|cy|>', '<|sk|>', '<|te|>', '<|fa|>', '<|lv|>', '<|bn|>', '<|sr|>', '<|az|>', '<|sl|>', '<|kn|>', '<|et|>', '<|mk|>', '<|br|>', '<|eu|>', '<|is|>', '<|hy|>', '<|ne|>', '<|mn|>', '<|bs|>', '<|kk|>', '<|sq|>', '<|sw|>', '<|gl|>', '<|mr|>', '<|pa|>', '<|si|>', '<|km|>', '<|sn|>', '<|yo|>', '<|so|>', '<|af|>', '<|oc|>', '<|ka|>', '<|be|>', '<|tg|>', '<|sd|>', '<|gu|>', '<|am|>', '<|yi|>', '<|lo|>', '<|uz|>', '<|fo|>', '<|ht|>', '<|ps|>', '<|tk|>', '<|nn|>', '<|mt|>', '<|sa|>', '<|lb|>', '<|my|>', '<|bo|>', '<|tl|>', '<|mg|>', '<|as|>', '<|tt|>', '<|haw|>', '<|ln|>', '<|ha|>', '<|ba|>', '<|jw|>', '<|su|>', '<|translate|>', '<|transcribe|>', '<|startoflm|>', '<|startofprev|>', '<|nocaptions|>', '<|notimestamps|>']

dict={50257+i:special_tokens[i] for i in range(0,len(special_tokens))}
print(dict)

{50257: '<|endoftext|>', 50258: '<|startoftranscript|>', 50259: '<|en|>', 50260: '<|zh|>', 50261: '<|de|>', 50262: '<|es|>', 50263: '<|ru|>', 50264: '<|ko|>', 50265: '<|fr|>', 50266: '<|ja|>', 50267: '<|pt|>', 50268: '<|tr|>', 50269: '<|pl|>', 50270: '<|ca|>', 50271: '<|nl|>', 50272: '<|ar|>', 50273: '<|sv|>', 50274: '<|it|>', 50275: '<|id|>', 50276: '<|hi|>', 50277: '<|fi|>', 50278: '<|vi|>', 50279: '<|he|>', 50280: '<|uk|>', 50281: '<|el|>', 50282: '<|ms|>', 50283: '<|cs|>', 50284: '<|ro|>', 50285: '<|da|>', 50286: '<|hu|>', 50287: '<|ta|>', 50288: '<|no|>', 50289: '<|th|>', 50290: '<|ur|>', 50291: '<|hr|>', 50292: '<|bg|>', 50293: '<|lt|>', 50294: '<|la|>', 50295: '<|mi|>', 50296: '<|ml|>', 50297: '<|cy|>', 50298: '<|sk|>', 50299: '<|te|>', 50300: '<|fa|>', 50301: '<|lv|>', 50302: '<|bn|>', 50303: '<|sr|>', 50304: '<|az|>', 50305: '<|sl|>', 50306: '<|kn|>', 50307: '<|et|>', 50308: '<|mk|>', 50309: '<|br|>', 50310: '<|eu|>', 50311: '<|is|>', 50312: '<|hy|>', 50313: '<|ne|>', 50314: '

#### Getting Different Token ids for Different Tasks
- Right Now I've used For english language Transcribtion

In [5]:
import numpy as np
from transformers import AutoConfig, AutoProcessor


model = "openai/whisper-tiny"
config = AutoConfig.from_pretrained(model)
processor = AutoProcessor.from_pretrained(model)

# English transcription
forced_decoder_ids = processor.get_decoder_prompt_ids(language="en", task="transcribe")
print(forced_decoder_ids)
# forced_decoder_ids is of the format [(1, 50259), (2, 50359), (3, 50363)] and needs to be
# of the format [50258, 50259, 50359, 50363] where 50258 is the start token id
forced_decoder_ids = [config.decoder_start_token_id] + list(map(lambda token: token[1], forced_decoder_ids))
print(forced_decoder_ids)
# If you don't want to provide specific decoder input ids or you want
# Whisper to predict the output language and task, you can set
# forced_decoder_ids = [config.decoder_start_token_id]
# [50258]

# decoder input ids
decoder_input_ids = np.array([forced_decoder_ids], dtype=np.int32)
print(decoder_input_ids)

/usr/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[(1, 50259), (2, 50359), (3, 50363)]
[50258, 50259, 50359, 50363]
[[50258 50259 50359 50363]]


## Getting the tflite Model

### data preprocessing

In [6]:
from datasets import load_dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import os
import numpy as np

import torch

# loading the processor
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
!rm -rf input_features
os.makedirs('input_features',exist_ok=True)

# load dummy dataset and read audio files
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
print(ds)
for i in range(25):
    sample = ds[i]["audio"]
    input_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="np") 
    
    inp_val=input_features.input_features.astype(np.float32)
    #Need to transpose the input
    print(inp_val.shape)
    updated_inp_val=inp_val.transpose(0,2,1)
    print(updated_inp_val.shape)
    with open("input_features/inp_val_"+str(i)+".raw", 'wb') as f:
        updated_inp_val.tofile(f)


#Creating list.txt
with open("list.txt",'w') as f:
    for i in range(25):
        f.write("x.1:=input_features/inp_val_"+str(i)+".raw\n")

Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_rows: 73
})
(1, 80, 3000)
(1, 3000, 80)
(1, 80, 3000)
(1, 3000, 80)
(1, 80, 3000)
(1, 3000, 80)
(1, 80, 3000)
(1, 3000, 80)
(1, 80, 3000)
(1, 3000, 80)
(1, 80, 3000)
(1, 3000, 80)
(1, 80, 3000)
(1, 3000, 80)
(1, 80, 3000)
(1, 3000, 80)
(1, 80, 3000)
(1, 3000, 80)
(1, 80, 3000)
(1, 3000, 80)
(1, 80, 3000)
(1, 3000, 80)
(1, 80, 3000)
(1, 3000, 80)
(1, 80, 3000)
(1, 3000, 80)
(1, 80, 3000)
(1, 3000, 80)
(1, 80, 3000)
(1, 3000, 80)
(1, 80, 3000)
(1, 3000, 80)
(1, 80, 3000)
(1, 3000, 80)
(1, 80, 3000)
(1, 3000, 80)
(1, 80, 3000)
(1, 3000, 80)
(1, 80, 3000)
(1, 3000, 80)
(1, 80, 3000)
(1, 3000, 80)
(1, 80, 3000)
(1, 3000, 80)
(1, 80, 3000)
(1, 3000, 80)
(1, 80, 3000)
(1, 3000, 80)
(1, 80, 3000)
(1, 3000, 80)


##### Running the Decoder Model
- Take the last_hidden_state of the encoder model
- Then take the initial decoder_input_ids then one by one add


##### Tflite Decoder Block

In [8]:
import numpy as np
import torch
import tensorflow as tf
from transformers import (
        AutoTokenizer
    )
model_id = "openai/whisper-tiny"
tokenizer = AutoTokenizer.from_pretrained(model_id)


tflite_model_path='openai-whisper/models/whisper-decoder-tiny.tflite'
#tflite_model_path='/content/whisper-decoder_main-int8.tflite'
print(tflite_model_path)

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()



def decoder_block_tflite(encoder_hidden_states):
    
    decoder_input_ids = torch.tensor([50258, 50259, 50359, 50363])
    decoder_input_ids = tf.expand_dims(decoder_input_ids, 0)
    
    input_tensor_1 = interpreter.get_input_details()[0]['index']
    
    interpreter.set_tensor(input_tensor_1, encoder_hidden_states)
    
    input_tensor_2 = interpreter.get_input_details()[1]['index']
    interpreter.resize_tensor_input(input_tensor_2, decoder_input_ids.shape)
    # Allocate memory for input and output tensors
    interpreter.allocate_tensors()
    interpreter.set_tensor(input_tensor_2, decoder_input_ids)
    output_tensor = interpreter.get_output_details()[0]['index']
    start_tokens = [50258, 50259, 50359, 50363] 
    tokens = start_tokens
    while(True):
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_tensor) 
        # print(output_data.shape)
        # print(output_data)
        cleaned = np.argmax(output_data, axis=-1)
        # print("cleaned",cleaned)
        last_token = cleaned[0,-1]
        # print("Last Token",last_token)
        tokens.append(last_token)
        # print("Updated tokens:",tokens)
        new_value = tf.constant([last_token], dtype=tf.int64)
        new_value = tf.reshape(new_value, (1,1))
        decoder_input_ids = tf.concat([decoder_input_ids, new_value], axis=1)
        input_tensor_2 = interpreter.get_input_details()[1]['index']
        interpreter.resize_tensor_input(input_tensor_2, decoder_input_ids.shape)
        # Allocate memory for input and output tensors
        interpreter.allocate_tensors()
        interpreter.set_tensor(input_tensor_2, decoder_input_ids)
        if last_token == 50257:
          break
    
    
    return tokenizer.batch_decode(np.expand_dims(tokens, axis=0), skip_special_tokens=True)[0]
    



openai-whisper/models/whisper-decoder-tiny.tflite


INFO: Created TensorFlow Lite delegate for select TF ops.
2025-06-23 10:59:10.397146: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-23 10:59:10.409749: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /local/mnt/workspace/snpe/qairt/2.35.0.250530/lib/x86_64-linux-clang
2025-06-23 10:59:10.409765: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2025-06-23 10:59:10.409787: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (hu-vraidu-hyd): /proc/d

#### Getting the encoder DLC Model
- The above ONNX model is running fine
- Now Converting to DLC and checking how it's working

In [9]:
import os

os.environ['SNPE_ROOT']="/local/mnt/workspace/snpe/qairt/2.35.0.250530"

#### Getting the Fp32 Model

In [ ]:
!pip3 install protobuf==3.20.2

In [10]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh

snpe-onnx-to-dlc -i whisper_encoder.onnx -d x.1 1,80,3000 -o whisper_tiny_encoder_fp32.dlc
snpe-dlc-info -i whisper_tiny_encoder_fp32.dlc > whisper_tiny_encoder_fp32.txt
snpe-dlc-viewer -i whisper_tiny_encoder_fp32.dlc -s whisper_tiny_encoder_fp32.html 

[INFO] AISW SDK environment set
[INFO] QNN_SDK_ROOT: /local/mnt/workspace/snpe/qairt/2.35.0.250530
[INFO] SNPE_ROOT: /local/mnt/workspace/snpe/qairt/2.35.0.250530


2025-06-23 10:59:55,817 - 270 - INFO - Input shape info 


`overwrite_input_shapes` and/or `test_input_shapes` instead. An error will be 
raised in the future.


2025-06-23 11:00:01,253 - 270 - INFO - INFO_INITIALIZATION_SUCCESS: 
2025-06-23 11:00:01,415 - 270 - INFO - INFO_CONVERSION_SUCCESS: Conversion completed successfully
2025-06-23 11:00:01,496 - 270 - INFO - INFO_WRITE_SUCCESS: 


Network Model HTML file saved at /local/mnt/workspace/qidk_snpe/qidk/Solutions/NLPSolution3-AutomaticSpeechRecognition-Whisper/Generate_Assets/whisper_tiny_encoder_fp32.html


#### Creating w8a16 model
* Choose --htp_socs based on the end device where model will be deployed. Example sm8750 or sm8650 or sm8550
* --optimizations cle --axis_quant

In [11]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh

snpe-dlc-quantize --input_dlc whisper_tiny_encoder_fp32.dlc --input_list list.txt  --output_dlc whisper_tiny_encoder_w8a16.dlc --weights_bitwidth 8 --act_bitwidth 16 --enable_htp --htp_socs sm8750

snpe-dlc-info -i whisper_tiny_encoder_w8a16.dlc > whisper_tiny_encoder_w8a16.txt
snpe-dlc-viewer -i whisper_tiny_encoder_w8a16.dlc -s whisper_tiny_encoder_w8a16.html  

[INFO] AISW SDK environment set
[INFO] QNN_SDK_ROOT: /local/mnt/workspace/snpe/qairt/2.35.0.250530
[INFO] SNPE_ROOT: /local/mnt/workspace/snpe/qairt/2.35.0.250530


[INFO] InitializeStderr: DebugLog initialized.
[INFO] Processed command-line arguments
[INFO] Quantized parameters


     4.0ms [  INFO ] Inferences will run in sync mode
qnn-net-run log level is : QNN_LOG_LEVEL_INFO
Processing inference input(s):
x_1 -> input_features/inp_val_0.raw
x_1 -> input_features/inp_val_1.raw
x_1 -> input_features/inp_val_2.raw
x_1 -> input_features/inp_val_3.raw
x_1 -> input_features/inp_val_4.raw
x_1 -> input_features/inp_val_5.raw
x_1 -> input_features/inp_val_6.raw
x_1 -> input_features/inp_val_7.raw
x_1 -> input_features/inp_val_8.raw
x_1 -> input_features/inp_val_9.raw
x_1 -> input_features/inp_val_10.raw
x_1 -> input_features/inp_val_11.raw
x_1 -> input_features/inp_val_12.raw
x_1 -> input_features/inp_val_13.raw
x_1 -> input_features/inp_val_14.raw
x_1 -> input_features/inp_val_15.raw
x_1 -> input_features/inp_val_16.raw
x_1 -> input_features/inp_val_17.raw
x_1 -> input_features/inp_val_18.raw
x_1 -> input_features/inp_val_19.raw
x_1 -> input_features/inp_val_20.raw
x_1 -> input_features/inp_val_21.raw
x_1 -> input_features/inp_val_22.raw
x_1 -> input_features/inp_va

[INFO] Generated activations
[USER_INFO] Successfully saved DLC to /local/mnt/workspace/qidk_snpe/qidk/Solutions/NLPSolution3-AutomaticSpeechRecognition-Whisper/Generate_Assets/whisper_tiny_encoder_w8a16.dlc
[INFO] Saved quantized dlc to: whisper_tiny_encoder_w8a16.dlc
[INFO] DebugLog shutting down.


xecute end
  9308.4ms [  INFO ] cleaning up resources for input tensors
  9308.4ms [  INFO ] cleaning up resources for output tensors
  9313.4ms [  INFO ] [QNN_CPU] QnnGraph execute start
  9981.4ms [  INFO ] [QNN_CPU] QnnGraph execute end
  9977.6ms [  INFO ] cleaning up resources for input tensors
  9977.6ms [  INFO ] cleaning up resources for output tensors
  9982.6ms [  INFO ] [QNN_CPU] QnnGraph execute start
 10605.2ms [  INFO ] [QNN_CPU] QnnGraph execute end
 10601.5ms [  INFO ] cleaning up resources for input tensors
 10601.5ms [  INFO ] cleaning up resources for output tensors
 10606.5ms [  INFO ] [QNN_CPU] QnnGraph execute start
 11227.2ms [  INFO ] [QNN_CPU] QnnGraph execute end
 11223.4ms [  INFO ] cleaning up resources for input tensors
 11223.4ms [  INFO ] cleaning up resources for output tensors
 11228.5ms [  INFO ] [QNN_CPU] QnnGraph execute start
 11861.6ms [  INFO ] [QNN_CPU] QnnGraph execute end
 11857.8ms [  INFO ] cleaning up resources for input tensors
 11857.9ms [

[INFO] InitializeStderr: DebugLog initialized.
[INFO] SNPE HTP Offline Prepare: Attempting to create cache for SM8750
[USER_INFO] Target device backend record identifier: HTP_V79_SM8750_8MB
[USER_INFO] No cache record in the DLC matches the target device (HTP_V79_SM8750_8MB). Creating a new record
[USER_INFO] Checking unsigned PD session
[INFO] Attempting to open dynamically linked lib: libHtpPrepare.so
[INFO] dlopen libHtpPrepare.so SUCCESS handle 0x1a45acd0
[INFO] Found Interface Provider (v2.26)
[USER_WARNING]  <W> Initializing HtpProvider
[USER_WARNING]  <W> HTP arch will be deprecated, please set SoC id instead.
[USER_WARNING]  <W> Performance Estimates unsupported
[USER_INFO] Platform option not set
[USER_INFO] Created ctx=0x1 for Graph Id=0 backend=HTP SNPE Id=0x1a3d7238
[USER_INFO] Context [0x1] Setting priority to: default
[USER_INFO] Offline Prepare VTCM size(MB) selected = 0
[USER_INFO] Offline Prepare Optimization Level passed = 2
[USER_WARNING]  <W> Sanitizing the value fo


====== DDR bandwidth summary ======
spill_bytes=655663104
fill_bytes=764239872
write_total_bytes=664879104
read_total_bytes=774336512



[USER_INFO] Backend Mgr ~Dtor called for backend HTP
[USER_INFO] Cleaning up Context handle=0x1 for Graph Id=0 backend=HTP SNPE Id=0x1a3d7238
[USER_INFO] Done Cleaning up Context handle=0x1 for Graph Id=0 backend=HTP SNPE Id=0x1a3d7238
[USER_INFO] BackendTerminate triggered
[USER_INFO] Finalizing cache record: backend.metadata0
[INFO] SNPE HTP Offline Prepare: Successfully created cache for SM8750
[INFO] ======== Run Summary ========
[INFO]   SM8750 :  Success
[USER_INFO] Successfully saved DLC to /local/mnt/workspace/qidk_snpe/qidk/Solutions/NLPSolution3-AutomaticSpeechRecognition-Whisper/Generate_Assets/whisper_tiny_encoder_w8a16.dlc
[USER_INFO] BackendTerminate triggered
[INFO] DebugLog shutting down.


Network Model HTML file saved at /local/mnt/workspace/qidk_snpe/qidk/Solutions/NLPSolution3-AutomaticSpeechRecognition-Whisper/Generate_Assets/whisper_tiny_encoder_w8a16.html


### Inferencing the FP32 Model on linux x86 machine

In [12]:
%%bash
rm -rf OUTPUT_Encoder

In [13]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh

snpe-net-run --container whisper_tiny_encoder_fp32.dlc --input_list list.txt --output_dir OUTPUT_Encoder

[INFO] AISW SDK environment set
[INFO] QNN_SDK_ROOT: /local/mnt/workspace/snpe/qairt/2.35.0.250530
[INFO] SNPE_ROOT: /local/mnt/workspace/snpe/qairt/2.35.0.250530
Logging level is : SNPE_LOG_LEVEL_ERROR
-------------------------------------------------------------------------------
Model String: N/A
SNPE v2.35.0.250530123435_121478
-------------------------------------------------------------------------------

Processing graph : whisper_tiny_encoder_fp32
Processing DNN input(s):
input_features/inp_val_0.raw
Processing DNN input(s):
input_features/inp_val_1.raw
Processing DNN input(s):
input_features/inp_val_2.raw
Processing DNN input(s):
input_features/inp_val_3.raw
Processing DNN input(s):
input_features/inp_val_4.raw
Processing DNN input(s):
input_features/inp_val_5.raw
Processing DNN input(s):
input_features/inp_val_6.raw
Processing DNN input(s):
input_features/inp_val_7.raw
Processing DNN input(s):
input_features/inp_val_8.raw
Processing DNN input(s):
input_features/inp_val_9.raw


#### Checking the output

In [14]:
import glob
import tensorflow as tf
import os

folder = ["OUTPUT_Encoder"]


for j in range(0,1):
    for result_path in glob.glob(os.path.join(folder[j], '*')):
        if ".log" not in result_path:
            last_hidden_state = np.fromfile(result_path+'/599.raw', dtype="float32")
            
            encoder_hidden_states=last_hidden_state.reshape((1,1500,384))
            print(decoder_block_tflite(encoder_hidden_states))

            

 Painting, he tells us, is of a different quality to mathematics, and finish in art is adding more effect.
 by Harry Quilter MA.
 As for etchings, there are two kinds, British and foreign.
 In fact, he is quite severe on Mr. Ruskin, for not recognizing that a picture should denote the frailty of man, and remarks with pleasing courtesy and falseness graced that many faces are feeling
 I begged Ruggano a long ago to send him away, but he would not do so.
 He doesn't work at all.
 He has gone and gone for good, answered Paul Icrom, who had managed to squeeze into the room beside the dragon, and had witnessed the occurrences with much interest.
 Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel.
 The King is flooded in disgrace and your friends are asking for you.
 L'Inille's pictures are a sort of upguards and atom paintings, and Mason's exquisite itles are as national as a jingo poem. Mr. Berkett Foster's landscapes smile at one much in the same way 

## Inferencing on Device

In [ ]:
%%bash
adb devices

In [ ]:
import os
os.environ['SNPE_ROOT']="/local/mnt/workspace/snpe/2.29.0.241129" #set up your snpe path here.
os.environ['RAW_FILE_FOLDER']="input_features"
os.environ['FOLDER_WITH_ARTIFACTS']="whisper"
os.environ['DLCFP32']="whisper_tiny_encoder_fp32.dlc"
os.environ['DLCA8W16']="whisper_tiny_encoder_w8a16.dlc"
os.environ['DLCA8W8']="whisper_tiny_encoder_w8a8.dlc"
os.environ['DLCA16W16']="whisper_tiny_encoder_w16a16.dlc"
os.environ['TARGET_INPUT_LIST']="list.txt"
os.environ['ONDEVICE_FOLDER']="whisper"
os.environ['DEVICE_HOST']="localhost"
os.environ['DEVICE_ID']="58671ff7" #fill your device-id. Use command "adb devices" to get devices names. example :"e18d5d0"
os.environ['SNPE_TARGET_ARCH']="aarch64-android"
os.environ['SNPE_TARGET_STL']="libc++_shared.so"

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/bin" && $DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib" && $DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/dsp/lib"

* In below code block please use
    - v79 for sm8750
    - v75 for sm8650
    - v73 for sm8550

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL push $SNPE_ROOT/lib/$SNPE_TARGET_ARCH/$SNPE_TARGET_STL /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib
$DEVICE_SHELL push $SNPE_ROOT/bin/$SNPE_TARGET_ARCH/snpe-net-run /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/bin
$DEVICE_SHELL push $SNPE_ROOT/lib/hexagon-v79/unsigned/*.so /data/local/tmp/snpeexample/dsp/lib
$DEVICE_SHELL push $SNPE_ROOT/lib/$SNPE_TARGET_ARCH/*.so /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell "mkdir -p /data/local/tmp/$ONDEVICE_FOLDER"

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL push $DLCA8W16 /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $DLCFP32 /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push input_features /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $TARGET_INPUT_LIST /data/local/tmp/$ONDEVICE_FOLDER

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell
chmod -R 777 /data/local/tmp/snpeexample
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/data/local/tmp/snpeexample/aarch64-android/lib
export PATH=$PATH:/data/local/tmp/snpeexample/aarch64-android/bin
export OUTPUT_FOLDER=OUTPUT_32b_CPU
export OUTPUT_DLC_32=whisper_tiny_encoder_fp32.dlc
export ONDEVICE_FOLDER="whisper"
cd /data/local/tmp/$ONDEVICE_FOLDER &&
snpe-net-run --container $OUTPUT_DLC_32 --input_list list.txt   --output_dir $OUTPUT_FOLDER

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell
chmod -R 777 /data/local/tmp/snpeexample
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/data/local/tmp/snpeexample/aarch64-android/lib
export PATH=$PATH:/data/local/tmp/snpeexample/aarch64-android/bin
export ADSP_LIBRARY_PATH="/data/local/tmp/snpeexample/dsp/lib;/system/lib/rfsa/adsp;/system/vendor/lib/rfsa/adsp;/dsp"
export OUTPUT_FOLDER=OUTPUT_DSP_W8A16
export DLC_W8A16=whisper_tiny_encoder_w8a16.dlc
export ONDEVICE_FOLDER="whisper"
cd /data/local/tmp/$ONDEVICE_FOLDER &&
snpe-net-run --container $DLC_W8A16 --input_list list.txt  --output_dir $OUTPUT_FOLDER --use_dsp --enable_cpu_fallback

In [ ]:
%%bash
rm -rf OUTPUT_32b_CPU/
rm -rf OUTPUT_DSP_W8A16/

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL pull /data/local/tmp/$ONDEVICE_FOLDER/OUTPUT_DSP_W8A16 OUTPUT_DSP_W8A16
$DEVICE_SHELL pull /data/local/tmp/$ONDEVICE_FOLDER/OUTPUT_32b_CPU OUTPUT_32b_CPU

### Checking the Output

In [ ]:
import glob
import tensorflow as tf
import os
import numpy as np

folder = ["OUTPUT_32b_CPU","OUTPUT_DSP_W8A16"]


for j in range(0,2):
    print("------------------------------"+folder[j]+"------------------------------")
    for result_path in glob.glob(os.path.join(folder[j], '*')):
        if ".log" not in result_path:
            last_hidden_state = np.fromfile(result_path+'/599.raw', dtype="float32")
            
            encoder_hidden_states=last_hidden_state.reshape((1,1500,384))
            print(decoder_block_tflite(encoder_hidden_states))

            